In [1]:
import numpy as np
import pandas as pd
import time
from functools import partial

from scipy.stats import randint, uniform
from sim_lib import simulation

pd.options.mode.chained_assignment = None


In [2]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        return history.index[randint.rvs(0, n)]

    ctr = history['clicks'] / (history['impressions'] + 10)
    n = np.argmax(ctr)
    return history.index[n]

policy = partial(eps_greedy, eps=0.06)

In [3]:
# seed for homework
seed = 18475
np.random.seed(seed=seed)

start = time.time()
output = simulation(policy, n=200000, seed=seed)
end = time.time()
end - start

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


568.3456523418427

In [4]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(1540.7609683932544, 0.007703804841966272, 184)

In [5]:
output['history']

,impressions,clicks,lifetime,p
153,18970.0,4199.0,18003.025431,0.220134
162,228.0,26.0,1537.166719,0.113780
172,163.0,32.0,19648.592394,0.219968
173,170.0,18.0,12771.474990,0.122694
180,26.0,3.0,4655.819793,0.020061
182,6.0,0.0,889.624649,0.004621
183,1.0,0.0,15187.163761,0.073886


Реализуем UCB с возможностью подбора множителя перед корневой составляющей

In [4]:
class UCB:
    def __init__(self, C: float):
        self.counter = 0
        self.C = C

    def __call__(self, history: pd.DataFrame):
        self.counter += 1
        ctr = history['clicks'] / (history['impressions'] + 10) + self.C * np.sqrt(2 * np.log(self.counter) / (history['impressions'] + 10))
        n = np.argmax(ctr)
        return history.index[n]

Подбираем множитель

In [14]:
for C in [0.0001, 0.001, 0.01, 0.05]:
    np.random.seed(seed=seed)
    start = time.time()
    output = simulation(UCB(C=C), n=200000, seed=seed)
    end = time.time()
    print(f"C={C}, time={end - start}, metrics={output['regret'], output['regret']/output['rounds'],  output['total_banners']}")


1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
C=0.0001, time=768.5177705287933, metrics=(7570.250349143192, 0.03785125174571596, 184)
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulate

Подбираем ещё, как видно при `C=0.2`, `regret ~ 457`, что лучше, чем бейзлайн.

In [15]:
for C in [0.1, 0.2, 0.3, 0.75]:
    np.random.seed(seed=seed)
    start = time.time()
    output = simulation(UCB(C=C), n=200000, seed=seed)
    end = time.time()
    print(f"C={C}, time={end - start}, metrics={output['regret'], output['regret']/output['rounds'],  output['total_banners']}")

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
C=0.1, time=829.9235732555389, metrics=(2066.259605524522, 0.01033129802762261, 184)
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
4